In [60]:
import requests 
from bs4 import BeautifulSoup 
import re
import pandas as pd

In [61]:
#check data exist or not
def ckeck_exist(url,pages):
    if url in pages:
        return 0

In [62]:
# 發起請求
def start_requests(url):
    r = requests.get(url)
    if r.status_code == requests.codes.ok:
        return r.content
    else:
        return 0

In [63]:
# 解析一級網頁 獲取 url List
def get_page():
    pages = []
    for genre in range(1, 13):
        for page in range(1,90):
            url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id={}&page={}'.format(genre,page)
            text = start_requests(url)
            if text !=0:
                soup = BeautifulSoup(text, 'html.parser')
                movies = soup.find_all('div', class_ = 'release_info')

                for movie in movies:
                    url = movie.find('div', class_='release_movie_name').a['href']
                    tff = ckeck_exist(url,pages)
                    if tff != 0:
                        pages.append(url)
    for genre in range(13, 21):
            for page in range(1,21):
                url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id={}&page={}'.format(genre,page)
                text = start_requests(url)
                if text !=0:
                    soup = BeautifulSoup(text, 'html.parser')
                    movies = soup.find_all('div', class_ = 'release_info')

                    for movie in movies:
                        url = movie.find('div', class_='release_movie_name').a['href']
                        tff = ckeck_exist(url,pages)
                        if tff != 0:
                            pages.append(url)
        
    return pages

In [64]:
# 解析二級網頁 獲取訊息
def parse_page(url):
    soup = BeautifulSoup(url, 'html.parser')
    info = {}
    #categories = []
    category = {}
    info['中文名稱'] = soup.find('div',class_= 'movie_intro_info_r').find('h1').text.strip()
    info['英文名稱'] = soup.find('div',class_= 'movie_intro_info_r').find('h3').text.strip()
    info['上映日期'] = soup.find('div',class_= 'movie_intro_info_r').find('span').text.split('上映日期：')[1]

    for i,d in enumerate(soup.find('div', class_='level_name_box').find_all('div',class_='level_name')):
        category[i] = d.find('a', class_='gabtn').text.strip()

    #categories.append(category)
    info['類別'] = category
    info['劇情介紹'] = re.sub('\s+','', soup.find('div', class_='gray_infobox_inner').find('span').text.strip())
    return info

In [65]:
result_list = []

pageurls = get_page() # 解析一級頁面
        
for pageurl in pageurls: # 解析二級頁面
    page = start_requests(pageurl)
    mydict = parse_page(page)
    result_list.append(mydict)
#write data to csv
df = pd.DataFrame(result_list)
df.to_csv('movies.csv', encoding='utf_8_sig')

df

,中文名稱,英文名稱,上映日期,類別,劇情介紹
0,小小兵2：格魯的崛起,Minions: The Rise of Guru,2022-07-01,"{0: '動畫', 1: '動作', 2: '喜劇'}",今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，...
1,蝙蝠俠,The Batman,2022-03-04,"{0: '動作', 1: '犯罪'}",《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版...
2,魔比斯,Morbius,2022-01-16,"{0: '動作', 1: '劇情'}",★索尼與漫威繼《猛毒》後再一野心暗黑爭議之作★《蜘蛛人》宇宙正式成立全新神秘反派吸血鬼「魔比...
3,捍衛戰士：獨行俠,Top Gun: Maverick,2021-11-19,"{0: '動作', 1: '劇情'}",派拉蒙影業1986年推出的《捍衛戰士》，由湯姆克魯斯主演，是一齣以航空母艦與美國海軍戰鬥機武...
4,魔鬼剋星：未來世,Ghostbusters: Afterlife,2021-11-11,"{0: '動作', 1: '冒險', 2: '懸疑/驚悚', 3: '喜劇'}",★抓鬼始祖回歸！《魔鬼剋星未來世》2020年夏天重磅登場★《蟻人》保羅路德攜《怪奇物與》天才...
...,...,...,...,...,...
6039,2017高雄電影節,,2017-10-20,{0: '影展'},全台最具「奇幻個性」的高雄電影節，今年以「慾望之味」做為17周年的獻禮，首創並集結影史永難忘...
6040,2017第三屆臺中國際動畫影展,,2017-10-13,{0: '影展'},國內一年一度的動畫盛事「臺中國際動畫影展」邁入第三屆，作為臺灣少數以動畫為主題的影展，除了推...
6041,阿莫多瓦 in 80s 經典數位修復影展,,未定,{0: '影展'},80年代的西班牙，剛從威權專制進入民主自由的時代，一切藝術、文化、娛樂活動蓬勃發展。阿莫多瓦...
6042,極道千金,Triad Princess,2019-12-06,"{0: '戲劇', 1: '影集'}",《極道千金》故事敘述一個從小就在黑道爸爸庇蔭下長大的女孩安琪（劉奕兒飾演），長相甜美卻個性剽...
